In [ ]:
!pip install --upgrade flwr tensorflow

import flwr as fl
import tensorflow as tf
from google.colab import drive
import numpy as np
import os
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, BatchNormalization, LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.1/727.1 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 105.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.1/250.1 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: click
    Found existing installation: click 8.3.1
    Uninstalling click-8.3.1:
      Successfully uninstalled click-8.3.1
  Attempting uninstall: tensorboard
    Found existing i

In [ ]:
CLIENT_ID = 1

drive.mount('/content/drive', force_remount=True)
DRIVE_DATA_PATH = '***' #enter path to data folder
data_path = os.path.join(DRIVE_DATA_PATH, f'client_{CLIENT_ID}_data.npz')

print(f"--- This notebook is now CLIENT #{CLIENT_ID} ---")
with np.load(data_path) as data:
    x_train = data['x'].astype(np.float32)
print(f"Successfully loaded {len(x_train)} samples for this client.")

Mounted at /content/drive
--- This notebook is now CLIENT #1 ---
Successfully loaded 20000 samples for this client.


In [ ]:
LATENT_DIM = 100
IMG_SHAPE = (28, 28, 1)

def build_generator():
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape
    model = Sequential([
        Dense(256, input_dim=LATENT_DIM), LeakyReLU(alpha=0.2), BatchNormalization(momentum=0.8),
        Dense(512), LeakyReLU(alpha=0.2), BatchNormalization(momentum=0.8),
        Dense(1024), LeakyReLU(alpha=0.2), BatchNormalization(momentum=0.8),
        Dense(np.prod(IMG_SHAPE), activation='tanh'), Reshape(IMG_SHAPE)
    ], name="generator")
    return model

def build_discriminator():
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Flatten, Dense, LeakyReLU
    model = Sequential([
        Flatten(input_shape=IMG_SHAPE), Dense(512), LeakyReLU(alpha=0.2),
        Dense(256), LeakyReLU(alpha=0.2), Dense(1, activation='sigmoid')
    ], name="discriminator")
    return model

class GanClient(fl.client.NumPyClient):
    def __init__(self, x_train):
        if x_train.max() > 1.0: self.x_train = (x_train - 127.5) / 1.5
        else: self.x_train = x_train * 2.0 - 1.0
        if self.x_train.ndim == 3: self.x_train = np.expand_dims(self.x_train, axis=3)

        self.generator = build_generator()
        self.discriminator = build_discriminator()
        self.generator_optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)
        self.discriminator_optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)
        self.loss_fn = tf.keras.losses.BinaryCrossentropy()

    def get_parameters(self, config):
        return self.generator.get_weights()

    @tf.function
    def _train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]
        noise = tf.random.normal([batch_size, LATENT_DIM])

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_images = self.generator(noise, training=True)

            real_output = self.discriminator(real_images, training=True)
            fake_output = self.discriminator(generated_images, training=True)

            gen_loss = self.loss_fn(tf.ones_like(fake_output), fake_output)

            real_loss = self.loss_fn(tf.ones_like(real_output), real_output)
            fake_loss = self.loss_fn(tf.zeros_like(fake_output), fake_output)
            disc_loss = real_loss + fake_loss

        grads_gen = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        grads_disc = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(grads_gen, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(grads_disc, self.discriminator.trainable_variables))

    def fit(self, parameters, config):
        self.generator.set_weights(parameters)

        dataset = tf.data.Dataset.from_tensor_slices(self.x_train).shuffle(len(self.x_train)).batch(32)

        for image_batch in dataset:
            self._train_step(image_batch)

        return self.generator.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config):
        return 0.0, len(self.x_train), {}

In [ ]:

print(f"\n--- Starting Client #{CLIENT_ID} ---")
client = GanClient(x_train)

fl.client.start_numpy_client(
    server_address="***",  #enter server ip
    client=client
)
print(f"\n--- Client #{CLIENT_ID} has finished. ---")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
	Instead, use `flwr.client.start_client()` by ensuring you first call the `.to_client()` method as shown below: 
	flwr.client.start_client(
		server_address='<IP>:<PORT>',
		client=FlowerClient()


--- Starting Client #1 ---


DEBUG:flwr:ChannelConnectivity.READY
INFO :      
INFO:flwr:
INFO :      Received: train message de1b0114-7fff-4b69-9d40-243991b4387e
INFO:flwr:Received: train message de1b0114-7fff-4b69-9d40-243991b4387e
INFO :      Sent reply
INFO:flwr:Sent reply
INFO :      
INFO:flwr:
INFO :      Received: evaluate message 0ceeedb3-78bd-4c9e-8bed-2c44ea1933b5
INFO:flwr:Received: evaluate message 0ceeedb3-78bd-4c9e-8bed-2c44ea1933b5
INFO :      Sent reply
INFO:flwr:Sent reply
INFO :      
INFO:flwr:
INFO :      Received: train message b5087789-c362-4df9-ae6a-fd92eebc0c8a
INFO:flwr:Received: train message b5087789-c362-4df9-ae6a-fd92eebc0c8a
INFO :      Sent reply
INFO:flwr:Sent reply
INFO :      
INFO:flwr:
INFO :      Received: evaluate message 61c13ab9-504c-45cf-a1c0-994d99e3379e
INFO:flwr:Received: evaluate message 61c13ab9-504c-45cf-a1c0-994d99e3379e
INFO :      Sent reply
INFO:flwr:Sent reply
INFO :      
INFO:flwr:
INFO :      Received: train message 97a27792-e462-4e72-a8e4-a5cae490dcbb
INFO:f


--- Client #1 has finished. ---
